# Analytic Scheme of Maneuver Generator

This notebook uses analytic plans to generate an analytic scheme of maneuver.

## Background

As described in TC 3-12.2.4.1, "The Analytic Scheme of Maneuver is the plan to collect and analyze technical data to meet specific information requirements. It identifies what data to analyze, how to analyze it, and why it is being analyzed." The analytic scheme of maneuver, or ASOM, consists of the following components:

* **Priority information requirement**:
* **Indicator**:
* **Evidence**:
* **Data**:
* **NAI**:
* **Analytic**:



## Environment Setup

This section sets up the environment. It installs packages necessary to generate the analytic plans, imports modules, initializes helper functions, and finally defines global variables. This section also mounts Google Drive to the runtime and moves into the project folder.

### Install Packages

In [ ]:
#!pip install -U -q "google" 1> /dev/null

### Import Modules

In [ ]:
# from google.colab import userdata
# from google.colab import drive
import json
import os
import datetime
import pandas as pd
import re
from collections import defaultdict

### Initialize Helper Functions

The first function, `log`, logs a message to the console prepended with the current timestamp in the ISO8601 format.

In [ ]:
def log(message, end="\n", flush = True):
    """
    Logs a message to the console, prepended with the current timestamp
    in ISO 8601 format.

    Args:
    message (str): The string message to log.
    """

    # Access the global flag controlling verbosity
    global verbose

    # Get the current date and time
    current_time = datetime.datetime.now()

    # Format the timestamp in ISO 8601 format
    timestamp = current_time.isoformat()

    # Construct the final log string using an f-string for clean formatting
    log_string = f"[{timestamp}] {message}"

    # Print the log string to the console if logging is turned on (verbose = True)
    if (verbose == True):
        print(log_string, end = end, flush = flush)

The second function, `build_asom`, accepts a series of MITRE ATT&CK techniques as input and returns a collection of analytic plans that correspond to those techniques.

In [ ]:
import os
import json
import copy
import re
from pathlib import Path
from typing import Dict, List, Set, Any, Tuple

TACTIC_ID_PATTERN = re.compile(r'^(?P<tactic_id>[A-Z0-9]{2,}-?[A-Z0-9]{0,})')
# Technique IDs: MITRE (T#### or T####.###) OR D3-* style identifiers
TECHNIQUE_ID_PATTERN = re.compile(r'^(?P<tech_id>T\d{4}(?:\.\d{3})?|D3-[A-Z]+)')

def _normalize_tactic_key(tactic: str) -> Tuple[str, str]:
    """
    Accepts strings like:
        "TA0001 - Initial Access"
        "D3-D - Detect"
        "TA0001"
    Returns (tactic_id, tactic_name_or_empty).
    """
    if " - " in tactic:
        tid, name = tactic.split(" - ", 1)
        return tid.strip(), name.strip()
    return tactic.strip(), ""

def _normalize_technique_id(tech: str) -> str:
    """
    Extract technique ID from various accepted forms:
      "T1055"
      "T1055.009"
      "T1055.009 - Process Injection"
      "D3-NTA - Network Traffic Analysis"
      "D3-PM"
    Returns the normalized ID or empty string if not found.
    """
    m = TECHNIQUE_ID_PATTERN.match(tech.strip())
    return m.group("tech_id") if m else ""

def _load_json_safely(path: Path) -> Any:
    """
    Loads JSON; strips code fences if present.
    Returns parsed object or raises.
    """
    text = path.read_text(encoding="utf-8").strip()
    if text.startswith("```"):
        # Remove fenced code markers (``` or ```json)
        text = re.sub(r'^```(?:json)?\s*', '', text)
        text = re.sub(r'\s*```$', '', text)
    return json.loads(text)

def _is_new_schema_object(obj: dict) -> bool:
    required = {"information_requirement", "tactic_id", "tactic_name", "indicators"}
    return isinstance(obj, dict) and required.issubset(obj.keys())

def _filter_indicators(ir_obj: dict, allowed_ids: Set[str]) -> dict:
    """
    Return a *new copy* of ir_obj with indicators filtered to those whose technique_id
    is in allowed_ids. If allowed_ids is empty, returns ir_obj unchanged.
    """
    if not allowed_ids:
        return ir_obj  # no filtering requested

    new_obj = copy.deepcopy(ir_obj)
    new_indicators = [
        ind for ind in new_obj.get("indicators", [])
        if _normalize_technique_id(ind.get("technique_id", "")) in allowed_ids
    ]
    new_obj["indicators"] = new_indicators
    return new_obj

def build_asom(
    attack_chain: Dict[str, List[str]],
    directory: str | Path = ".",
    detect_filename: str = "D3-D - Detect.json",
    include_detect_first: bool = True,
    filter_indicators: bool = True,
    deduplicate: bool = True
) -> List[dict]:
    """
    Build an ASOM (list of IR objects in the *new* analytic plan schema) filtered
    by the provided attack_chain.

    Parameters
    ----------
    attack_chain : dict
        Mapping of tactic strings -> list of technique strings.
        Tactic strings can be 'TA0001 - Initial Access' or just 'TA0001'.
        Techniques can be:
            'T1078'
            'T1078 - Valid Accounts'
            'T1055.009'
            'T1055.009 - Process Injection'
            'D3-NTA - Network Traffic Analysis'
            'D3-PM'
    directory : str | Path
        Directory containing new-format analytic plan JSON files.
    detect_filename : str
        Special file whose IR objects should be placed first (if include_detect_first = True).
    include_detect_first : bool
        If True, IR objects from detect_filename (if present) are prepended before the rest.
    filter_indicators : bool
        If True, restrict the 'indicators' list in each returned IR object to only those
        techniques explicitly requested for that tactic. If False, include all indicators
        for matching IR objects (as long as tactic matches).
    deduplicate : bool
        If True, remove duplicates (same information_requirement + tactic_id combination).
        Keeps the first occurrence (maintaining order).

    Returns
    -------
    list[dict]
        List of IR objects (each already in the new schema).
    """

    directory = Path(directory)

    # Normalize attack_chain into:
    #   tactic_map: tactic_id -> set(normalized technique IDs)
    tactic_map: Dict[str, Set[str]] = {}
    for tactic_str, technique_list in attack_chain.items():
        tactic_id, _ = _normalize_tactic_key(tactic_str)
        norm_tecs = {_normalize_technique_id(t) for t in technique_list}
        # Remove empties
        norm_tecs = {t for t in norm_tecs if t}
        tactic_map.setdefault(tactic_id, set()).update(norm_tecs)

    results: List[dict] = []

    def process_file(path: Path):
        try:
            data = _load_json_safely(path)
        except Exception as e:  # noqa
            print(f"[WARN] Could not parse {path.name}: {e}")
            return

        if not isinstance(data, list):
            # print(f"[WARN] {path.name} root is not a list. Skipping.")
            return

        for obj in data:
            if not _is_new_schema_object(obj):
                print(f"[WARN] {path.name} does not conform to schema. Skipping.")
                continue  # silently skip non-conforming entries

            tactic_id = obj.get("tactic_id", "").strip()
            if tactic_id not in tactic_map:
                # print(f"[INFO] {tactic_id} not in tactic map. Skipping.")
                continue  # tactic not requested

            # Possibly filter indicators
            if filter_indicators:
                filtered_obj = _filter_indicators(copy.deepcopy(obj), tactic_map[tactic_id])
                # If after filtering there are no indicators (and we requested some), skip.
                if tactic_map[tactic_id] and not filtered_obj.get("indicators"):
                    continue
                results.append(filtered_obj)
            else:
                results.append(copy.deepcopy(obj))

    # 1. Optionally process the special detect_filename first
    if include_detect_first:
        detect_path = directory / detect_filename
        if detect_path.exists():
            process_file(detect_path)
        else:
            print(f"[ERROR] {detect_filename} not found")
            pass

    # 2. Process all other JSON files
    for path in sorted(directory.glob("*.json")):
        if include_detect_first and path.name == detect_filename:
            continue
        process_file(path)

    # 3. Deduplicate (stable) if requested
    if deduplicate:
        seen = set()
        unique: List[dict] = []
        for obj in results:
            key = (obj.get("information_requirement"), obj.get("tactic_id"))
            if key in seen:
                continue
            seen.add(key)
            unique.append(obj)
        results = unique

    return results

### Define Global Variables

In [ ]:
# Toggle logging on (verbose = True)/off (verbose = False)
verbose = True
# verbose = False

In [ ]:
# Rate limits: https://ai.google.dev/gemini-api/docs/rate-limits
# Pricing: https://ai.google.dev/gemini-api/docs/pricing
# Usage: https://console.cloud.google.com/apis/api/generativelanguage.googleapis.com/metrics?project=gen-lang-client-0497172401
# Note that this notebook is designed to be run in Google Colab. The line below reads the Gemini API key for AI Studio,
# which is configured in the Secrets tab on the left side of the Colab window.
# os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")
# log("Gemii API key loaded.")

### Mount Google Drive

In [ ]:
# Mount Google Drive and move into the Google AI Studio folder
# DRIVE_PATH = "/content/drive"
# TECHNIQUES_PATH = "/content/drive/MyDrive/Google AI Studio/techniques"

# drive.mount(DRIVE_PATH)
# log(f"Google Drive mounted to {DRIVE_PATH}")

# os.chdir(TECHNIQUES_PATH)
# log(f"Changed directory to {TECHNIQUES_PATH}")

## Test Analytic Scheme of Maneuver Generation

This section generates a small analytic scheme of maneuver off of a test attack chain data set.

In [ ]:
# Example attack_chain data structure
attack_chain_data = {
    "D3-D - Detect": [
        "D3-NTA - Network Traffic Analysis",
        "D3-PM - Platform Monitoring"
    ],
    "TA0001 - Initial Access": [
        "T1190 - Exploit Public-Facing Application",
        "T1566 - Phishing",
        "T1078 - Valid Accounts"
    ],
    "TA0004 - Privilege Escalation": [
        "T1078 - Valid Accounts"
    ]
}

log("Building ASOM...")
resulting_asom = build_asom(attack_chain_data)
log("Finished.")

In [ ]:
# print(json.dumps(resulting_asom, indent=2))

In [ ]:
def format_asom(asom_input_list, joiner="; "):
    """
    Format the output from build_asom_new (new analytic plan schema) into a pandas DataFrame.

    Parameters
    ----------
    asom_input_list : list[dict]
        List of IR objects, each with keys:
          - information_requirement (str)
          - tactic_id (str)
          - tactic_name (str)
          - indicators (list of indicator dicts)
              * indicator dict: { "technique_id", "name", "evidence": [ evidence dicts ] }
              * evidence dict: { "description", "data_sources", "data_platforms", "nai", "action" }
          - version
          - date_created
          - last_updated
          - contributors (list)
    joiner : str
        Delimiter to join list fields (data_sources, data_platforms, contributors).

    Returns
    -------
    pandas.DataFrame
        Tabular view with hierarchical-style indices expressed as string columns:
          IR Index -> Indicator Index (IRIndex.TechSubIndex) -> Evidence Index (IRIndex.TechSubIndex.EvidenceSubIndex)
    """
    import pandas as pd

    table_rows = []
    ir_index = 0

    if not isinstance(asom_input_list, list):
        raise TypeError("Expected asom_input_list to be a list produced by build_asom_new.")

    for ir_obj in asom_input_list:
        if not isinstance(ir_obj, dict):
            print(f"[WARN] Skipping non-dict IR object: {ir_obj}")
            continue

        required_ir_keys = {"information_requirement", "tactic_id", "tactic_name", "indicators"}
        if not required_ir_keys.issubset(ir_obj.keys()):
            print(f"[WARN] IR object missing required keys: {ir_obj.keys()}")
            continue

        indicators = ir_obj.get("indicators", [])
        if not indicators:
            # Optionally emit a placeholder row; for now just warn and skip.
            print(f"[INFO] IR '{ir_obj.get('information_requirement')}' has no indicators; skipping.")
            continue

        ir_index += 1
        tactic_id = ir_obj.get("tactic_id", "")
        tactic_name = ir_obj.get("tactic_name", "")
        information_requirement = ir_obj.get("information_requirement", "")
        version = ir_obj.get("version", "")
        date_created = ir_obj.get("date_created", "")
        last_updated = ir_obj.get("last_updated", "")
        contributors = ir_obj.get("contributors", [])
        contributors_joined = joiner.join(contributors) if isinstance(contributors, list) else str(contributors)

        tech_sub_index = 0
        for indicator in indicators:
            if not isinstance(indicator, dict):
                print(f"[WARN] Skipping non-dict indicator in IR '{information_requirement}': {indicator}")
                continue

            technique_id = indicator.get("technique_id", "")
            technique_name = indicator.get("name", "")
            evidence_list = indicator.get("evidence", [])

            information_requirement = f"{information_requirement} ({tactic_id} - {tactic_name})"

            tech_sub_index += 1
            indicator_index_str = f"{ir_index}.{tech_sub_index}"

            if not evidence_list:
                # If no evidence entries, optionally create a placeholder row.
                # (Current behavior: skip but warn.)
                print(f"[INFO] Indicator '{technique_id} - {technique_name}' has no evidence entries.")
                continue

            evidence_sub_index = 0
            for evidence in evidence_list:
                if not isinstance(evidence, dict):
                    print(f"[WARN] Skipping non-dict evidence under technique '{technique_id}': {evidence}")
                    continue

                evidence_sub_index += 1
                evidence_index_str = f"{indicator_index_str}.{evidence_sub_index}"

                description = evidence.get("description", "")
                data_sources = evidence.get("data_sources", [])
                data_platforms = evidence.get("data_platforms", [])
                nai = evidence.get("nai", "")
                action = evidence.get("action", "")

                # Normalize list fields
                if isinstance(data_sources, list):
                    data_sources_joined = joiner.join(data_sources)
                else:
                    data_sources_joined = str(data_sources)

                if isinstance(data_platforms, list):
                    data_platforms_joined = joiner.join(data_platforms)
                else:
                    data_platforms_joined = str(data_platforms)

                row = {
                    "CCIR Index": ir_index,
                    "CCIR": information_requirement,
                    "Tactic ID": tactic_id,
                    "Tactic Name": tactic_name,
                    "Indicator Index": indicator_index_str,
                    "Indicator": f"{technique_id} - {technique_name}",
                    "Technique ID": technique_id,
                    "Technique Name": technique_name,
                    "Evidence Index": evidence_index_str,
                    "Evidence Description": description,
                    "Data Sources": data_sources_joined,
                    "Data Platforms": data_platforms_joined,
                    "NAI": nai,
                    "Action": action
                    # "Version": version,
                    # "Date Created": date_created,
                    # "Last Updated": last_updated,
                    # "Contributors": contributors_joined,
                }
                table_rows.append(row)

    df = pd.DataFrame(table_rows)

    # Ensure all expected columns exist (important if there were zero rows)
    column_order = [
        "CCIR Index",
        "CCIR",
        "Tactic ID",
        "Tactic Name",
        "Indicator Index",
        "Indicator",
        "Technique ID",
        "Technique Name",
        "Evidence Index",
        "Evidence Description",
        "Data Sources",
        "Data Platforms",
        "NAI",
        "Action"
        # "Version",
        # "Date Created",
        # "Last Updated",
        # "Contributors",
    ]
    for col in column_order:
        if col not in df.columns:
            df[col] = pd.NA

    # Optional: sort by hierarchical indices for readability
    if not df.empty:
        df.sort_values(by=["CCIR Index", "Indicator Index", "Evidence Index"], inplace=True, ignore_index=True)

    return df[column_order]


In [ ]:
formatted_df = format_asom(resulting_asom)
# To display the full content of cells if they are long
with pd.option_context('display.max_rows', None,
                        'display.max_columns', None,
                        'display.width', 1000,
                        'display.max_colwidth', None):
    display(formatted_df)

In [ ]:
# Export the full ASOM to an Excel file
formatted_df.to_excel("test_asom_full.xlsx")

In [ ]:
def get_representative_ccirs_by_tactic_simplified(df_with_ccirs: pd.DataFrame) -> dict:
    """
    Groups CCIRs by MITRE Tactic and generates a representative CCIR string for each group
    using a simplified templating logic.

    Args:
        df_with_ccirs (pd.DataFrame): DataFrame containing a 'CCIR' column with the
                                     CCIR strings.

    Returns:
        dict: A dictionary mapping each Tactic ID (str) to its representative
              CCIR string (str).
    """
    if 'CCIR' not in df_with_ccirs.columns:
        print("Error: DataFrame must contain a 'CCIR' column.")
        return {}

    df = df_with_ccirs.copy()

    def extract_tactic_details_from_ccir(ccir_string: str):
        """
        Extracts Tactic ID, Tactic Name, and the full original signature from a CCIR string.
        """
        if pd.isna(ccir_string):
            return None, None, None  # tactic_id, tactic_name, original_signature

        # Regex to capture "(TAXXXX - Tactic Name)"
        match_full = re.search(r"\((TA\d{4})\s*-\s*([^)]+?)\)", ccir_string)
        if match_full:
            tactic_id = match_full.group(1)      # e.g., TA0001
            tactic_name = match_full.group(2).strip()  # e.g., Initial Access
            original_signature = match_full.group(0) # e.g., (TA0001 - Initial Access)
            return tactic_id, tactic_name, original_signature

        # Fallback for patterns like "(TAXXXX)" if name is missing in signature
        match_simple = re.search(r"\((TA\d{4})\)", ccir_string)
        if match_simple:
            tactic_id = match_simple.group(1)
            original_signature = match_simple.group(0)
            # Tactic name is not present in this signature type
            return tactic_id, None, original_signature

        return None, None, None

    # Apply extraction to get Tactic ID, Tactic Name, and original signature
    extracted_info = df['CCIR'].apply(
        lambda x: pd.Series(extract_tactic_details_from_ccir(x),
                            index=['Tactic_ID_Extracted', 'Tactic_Name_Extracted', 'Original_Signature'])
    )
    df = pd.concat([df, extracted_info], axis=1)

    representative_ccirs_map = {}

    # Group by extracted Tactic ID
    # Consider only rows where Tactic_ID_Extracted is not NaN
    valid_tactic_groups = df[df['Tactic_ID_Extracted'].notna()].groupby('Tactic_ID_Extracted')

    for tactic_id, group in valid_tactic_groups:
        unique_ccirs = group['CCIR'].unique().tolist()

        # Attempt to get a consistent Tactic Name for the group
        # Prioritize non-null Tactic Names if there's variation (shouldn't be if signature is consistent)
        tactic_names_in_group = group['Tactic_Name_Extracted'].dropna().unique()
        tactic_name_for_template = tactic_names_in_group[0] if len(tactic_names_in_group) > 0 else None

        if len(unique_ccirs) == 1:
            representative_ccirs_map[tactic_id] = unique_ccirs[0]
        else:  # More than one unique CCIR for this Tactic ID
            if tactic_name_for_template: # Check if we have a tactic name for the template
                lowercase_tactic_name = tactic_name_for_template.lower()
                # Construct the standard signature part for the template
                standard_signature_for_template = f"({tactic_id} - {tactic_name_for_template})"
                templated_ccir = f"Did the adversary conduct {lowercase_tactic_name}? {standard_signature_for_template}"
                representative_ccirs_map[tactic_id] = templated_ccir
            else:
                # Fallback if Tactic Name is not available for the template, use the first unique CCIR
                representative_ccirs_map[tactic_id] = unique_ccirs[0]
                print(f"Warning: Tactic Name not found for Tactic ID {tactic_id} with multiple CCIRs. Using original CCIR as representative.")

    return representative_ccirs_map

In [ ]:
representative_ccirs_map = get_representative_ccirs_by_tactic_simplified(formatted_df)

In [ ]:
def update_ccirs(original_df, representative_ccirs_map):
    """
    Replaces CCIR strings in a DataFrame with their generalized versions.

    Args:
        original_df (pd.DataFrame): The DataFrame with an 'CCIR' column
                                     containing original CCIR strings.
        representative_ccirs_map (dict): A dictionary mapping Tactic IDs (e.g., 'TA0001')
                                        to their generalized CCIR strings.

    Returns:
        final_df (pd.DataFrame): The DataFrame with updated CCIRs.
    """
    if 'CCIR' not in original_df.columns:
        print("Error: DataFrame must contain an 'CCIR' column.")
        return original_df, None

    final_df = original_df.copy()

    # Helper function to extract Tactic ID from a CCIR string
    def extract_tactic_id(ccir_string):
        if pd.isna(ccir_string):
            return None
        # This regex aims to find (TAXXXX) within the string
        match = re.search(r"\((TA\d{4}).*?\)", ccir_string)
        if match:
            return match.group(1) # Returns TAXXXX
        return None

    # 1. Extract Tactic ID from original CCIRs to use for mapping
    final_df['Tactic_ID_for_mapping'] = final_df['CCIR'].apply(extract_tactic_id)

    # 2. Replace the 'CCIR' column with the generalized CCIRs
    # The .map() function looks up each 'Tactic_ID_for_mapping' in the representative_ccirs_map
    # .fillna(final_df['CCIR']) ensures that if a Tactic ID doesn't have a corresponding
    # generalized CCIR in the map, the original CCIR string is retained.
    final_df['CCIR'] = final_df['Tactic_ID_for_mapping'].map(representative_ccirs_map).fillna(final_df['CCIR'])

    # Clean up the temporary mapping column
    final_df = final_df.drop(columns=['Tactic_ID_for_mapping'])

    return final_df


In [ ]:
# Perform the update and grouping
updated_df = update_ccirs(formatted_df, representative_ccirs_map)

In [ ]:
def update_sequential_ccir_index(input_df: pd.DataFrame) -> pd.DataFrame:
    """
    Updates the 'CCIR Index' column in the DataFrame to sequentially number
    groups of adjacent identical strings in the 'CCIR' column.

    The DataFrame should already have its 'CCIR' column populated with the
    strings that need to be grouped (e.g., generalized CCIRs).
    Any existing 'CCIR Index' column will be overwritten.

    Args:
        input_df (pd.DataFrame): The DataFrame with a 'CCIR' column.

    Returns:
        pd.DataFrame: The DataFrame with the 'CCIR Index' column updated.
    """
    if 'CCIR' not in input_df.columns:
        print("Error: DataFrame must contain a 'CCIR' column to generate the new CCIR Index.")
        # Return a copy or original df if CCIR column is missing
        return input_df.copy() if isinstance(input_df, pd.DataFrame) else input_df


    df = input_df.copy()

    # Ensure consistent handling of NaNs if they exist in CCIR column,
    # though ideally CCIRs are strings. For this logic, direct comparison works.
    # If NaNs need specific grouping (e.g., all NaNs are one group, or each is distinct),
    # df['CCIR'] = df['CCIR'].fillna('__NAN_PLACEHOLDER__') # might be one strategy before comparison.
    # However, the current logic (ccir_series != ccir_series.shift()) handles NaNs by
    # typically starting a new group for/after a NaN, as NaN != NaN is True.

    # Identify rows where the 'CCIR' value changes from the previous row.
    # The first row will always mark a change (as shift() produces NaN).
    ccir_value_changed = (df['CCIR'] != df['CCIR'].shift())

    # The cumsum() of this boolean series creates a unique sequential ID for each
    # block of adjacent identical CCIRs. True becomes 1, False becomes 0 in sum.
    # This will naturally start the 'CCIR Index' from 1.
    df['CCIR Index'] = ccir_value_changed.cumsum()

    return df

In [ ]:
def update_hierarchical_indicator_index(input_df: pd.DataFrame,
                                        ccir_index_col: str = 'CCIR Index',
                                        indicator_text_col: str = 'Indicator', # Updated default
                                        output_indicator_index_col: str = 'Indicator Index') -> pd.DataFrame:
    """
    Updates or creates a hierarchical 'Indicator Index' column in the DataFrame.
    The index is in the format 'CCIR Index.Sub-Index'. The Sub-Index restarts
    for each new 'CCIR Index' group and increments for blocks of adjacent
    identical values in the 'indicator_text_col' within that CCIR Index group.

    Args:
        input_df (pd.DataFrame): The input DataFrame.
        ccir_index_col (str): Name of the column containing the main CCIR group index.
        indicator_text_col (str): Name of the column containing the indicator text
                                  strings (e.g., "T1190 - Exploit Public-Facing Application")
                                  to be grouped for sub-indexing. Default is 'Indicator'.
        output_indicator_index_col (str): Name of the column where the new
                                          hierarchical indicator index will be stored.
                                          Default is 'Indicator Index'.

    Returns:
        pd.DataFrame: The DataFrame with the 'Indicator Index' column updated or created.
    """
    df = input_df.copy()

    if ccir_index_col not in df.columns:
        print(f"Error: DataFrame must contain the CCIR group index column '{ccir_index_col}'.")
        return df
    if indicator_text_col not in df.columns:
        print(f"Error: DataFrame must contain the indicator text column '{indicator_text_col}'.")
        return df

    # --- Data Cleaning & Preparation for 'indicator_text_col' ---
    # This is crucial if strings that should group together have subtle differences
    # (e.g., extra whitespace, case differences if they should be ignored).
    processed_indicator_text_col = '_processed_' + indicator_text_col

    # Convert to string and strip whitespace as a basic cleaning step.
    # NaNs become "nan", "None" becomes "None". If specific NaN handling is needed,
    # it should be done here (e.g., fillna with a placeholder).
    df[processed_indicator_text_col] = df[indicator_text_col].astype(str).str.strip()
    # Example for case-insensitive grouping (optional):
    # df[processed_indicator_text_col] = df[indicator_text_col].astype(str).str.strip().str.lower()


    # Helper function to calculate the sub-index within each CCIR Index group
    def calculate_sub_index_within_group(indicator_series: pd.Series) -> pd.Series:
        """
        For a series of indicator texts within a single CCIR Index group,
        this calculates a sub-index (1, 2, 3...) for blocks of identical indicators.
        """
        indicator_changed_in_group = (indicator_series != indicator_series.shift())
        return indicator_changed_in_group.cumsum()

    temp_sub_index_col = '_temp_indicator_sub_index'

    # Apply sub-index calculation to each group formed by 'CCIR Index',
    # using the (potentially processed) 'indicator_text_col'.
    df[temp_sub_index_col] = df.groupby(ccir_index_col)[processed_indicator_text_col].transform(calculate_sub_index_within_group)

    # Construct the final hierarchical 'Indicator Index' string
    df[output_indicator_index_col] = df[ccir_index_col].astype(str) + '.' + df[temp_sub_index_col].astype(str)

    # Drop temporary columns
    df = df.drop(columns=[temp_sub_index_col, processed_indicator_text_col])

    return df

In [ ]:
def update_final_evidence_index(input_df: pd.DataFrame,
                                indicator_index_col: str = 'Indicator Index',
                                output_evidence_index_col: str = 'Evidence Index') -> pd.DataFrame:
    """
    Updates or creates the 'Evidence Index' column in the DataFrame.
    The index is hierarchical, in the format 'Indicator Index.n'
    (e.g., "1.1.1", "1.1.2"), where 'n' is the 1-based sequential
    position of the evidence row within its parent 'Indicator Index' group.

    Args:
        input_df (pd.DataFrame): The input DataFrame which must contain the
                                 column specified by `indicator_index_col`.
                                 Each row is assumed to be a distinct piece of evidence.
        indicator_index_col (str): Name of the column containing the hierarchical
                                   indicator index (e.g., a column with values
                                   like "1.1", "1.2" from the previous step).
        output_evidence_index_col (str): Name of the column where the new,
                                         most granular evidence index will be stored.

    Returns:
        pd.DataFrame: The DataFrame with the 'Evidence Index' column updated or created.
    """
    df = input_df.copy()

    if indicator_index_col not in df.columns:
        print(f"Error: DataFrame must contain the '{indicator_index_col}' column to generate Evidence Index.")
        return df

    # Calculate 'n', the sequential position of each evidence item (row)
    # within its group defined by the 'indicator_index_col'.
    # pandas.core.groupby.GroupBy.cumcount() is 0-based, so we add 1 for a 1-based sequence.
    # This assumes each row at this stage is a unique piece of evidence to be numbered.
    df['evidence_sequence_n'] = df.groupby(indicator_index_col).cumcount() + 1

    # Construct the final 'Evidence Index' string by appending '.n'
    # to the existing 'Indicator Index' string.
    df[output_evidence_index_col] = df[indicator_index_col].astype(str) + '.' + df['evidence_sequence_n'].astype(str)

    # Drop the temporary sequence helper column
    df = df.drop(columns=['evidence_sequence_n'])

    return df

In [ ]:
# Now that we have re-written the CCIR names, update the CCIR indexes .
updated_df = update_sequential_ccir_index(updated_df)

In [ ]:
# Next, update the indicator index based on the new CCIR indexes.
updated_df = update_hierarchical_indicator_index(updated_df)

In [ ]:
# Finally, update the evidence index based on the new CCIR and indicator indexes.
updated_df = update_final_evidence_index(updated_df, indicator_index_col = "Indicator Index", output_evidence_index_col = "Evidence Index")
# display(updated_df)

In [ ]:
def create_visually_spanned_df(input_df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforms a DataFrame by setting a MultiIndex on specified hierarchical
    columns to achieve a visual row-spanning effect when displayed.

    The "merging" of cells is a visual effect provided by Pandas' MultiIndex
    display. The underlying DataFrame data remains in a 2D structure.

    Args:
        input_df (pd.DataFrame): The input DataFrame. Expected to have columns
                                 like 'CCIR Index', 'CCIR', 'Tactic ID',
                                 'Technique Index', 'Technique', 'Technique ID',
                                 followed by more granular data columns.

    Returns:
        pd.DataFrame: A new DataFrame ('final_df') with a MultiIndex set on the
                      hierarchical columns, sorted for proper visual spanning.
    """
    df = input_df.copy()

    # Define the columns that will form the hierarchical MultiIndex.
    # The order of these columns is crucial as it defines the levels of the hierarchy
    # and how the visual spanning will appear.
    hierarchical_cols = [
        'CCIR Index',
        'CCIR',
        'Indicator Index',
        'Indicator',
    ]

    # Verify that these columns exist in the input DataFrame.
    # If any are missing, they will be excluded from the MultiIndex.
    existing_hierarchical_cols = [col for col in hierarchical_cols if col in df.columns]

    if not existing_hierarchical_cols:
        print("Warning: None of the specified hierarchical columns for indexing "
              "were found in the DataFrame. Returning the original DataFrame.")
        return df

    # The remaining columns will be the data columns associated with the
    # most granular level of the hierarchy.
    # data_cols = [col for col in df.columns if col not in existing_hierarchical_cols] # Not strictly needed for set_index

    # For the visual spanning to work correctly, the DataFrame MUST be sorted
    # by the columns that will form the MultiIndex, in the specified order.
    # This ensures that identical values are adjacent before set_index is called.
    df = df.sort_values(by=existing_hierarchical_cols)

    # Set the MultiIndex.
    # The columns listed in existing_hierarchical_cols will be moved from
    # the DataFrame's columns to its index.
    final_df = df.set_index(existing_hierarchical_cols)

    # While sort_values before set_index is the primary sorting,
    # sorting the index itself can ensure canonical order if needed,
    # though usually redundant if sorted before.
    # final_df = final_df.sort_index()

    return final_df

In [ ]:
# Now we can perform the final transformation on the ASOM, merging like cells to
# produce the finished ASOM.
final_df = create_visually_spanned_df(updated_df)

with pd.option_context('display.max_rows', None, # Show all rows to see full effect
                        'display.width', 1200,    # Adjust width as needed
                        'display.max_colwidth', 30,
                        'display.expand_frame_repr', False # Prevent wrapping if too wide for console
                        ):
    # display(final_df)
    # Here we will display only a specific subset of the columns, rather than the full set.
    display(final_df.filter(items=["CCIR Index", "CCIR", "Indicator Index", "Indicator ", "Evidence Index", "Evidence", "Data", "NAI", "Action"]))

In [ ]:
# Export the finished ASOM to an Excel file
final_df.to_excel("test_asom_merged.xlsx")

In [ ]:
# Automatically halt execution after the test ASOM is generated
sys.exit(0)

## Example Analytic Scheme of Maneuver Generation

This section generates an example analytic scheme of maneuver for a real threat actor based on information from Crowdstrike's threat intelligence platform.

In [ ]:
def generate_attack_chain_data(actor_data_list):
    """
    Transforms a list of actor data objects into a nested dictionary
    organizing techniques under tactics.

    Args:
        actor_data_list (list): A list of dictionaries, where each dictionary
                                represents an entry with tactic and technique info.

    Returns:
        dict: A dictionary where keys are formatted tactic strings
              (e.g., "TA0001 - Initial Access") and values are lists of
              formatted technique strings (e.g., "T1078 - Default Accounts").
    """
    attack_chain_data = defaultdict(list)

    if not isinstance(actor_data_list, list):
        print("Error: Input actor_data must be a list.")
        return {}

    for item in actor_data_list:
        if not isinstance(item, dict):
            print(f"Warning: Skipping non-dictionary item in actor_data_list: {item}")
            continue

        tactic_id = item.get('tactic_id')
        tactic_name = item.get('tactic_name')
        technique_id_full = item.get('technique_id')
        technique_name = item.get('technique_name')

        # Ensure all necessary fields are present
        if not all([tactic_id, tactic_name, technique_id_full, technique_name]):
            print(f"Warning: Skipping item due to missing essential fields (tactic_id, tactic_name, technique_id, technique_name): {item}")
            continue

        # Format the Tactic Key: "TAXXXX - Tactic Name"
        try:
            formatted_tactic_id = str(tactic_id).upper()
            tactic_key = f"{formatted_tactic_id} - {str(tactic_name)}"
        except Exception as e:
            print(f"Warning: Could not format tactic key for item {item}. Error: {e}")
            continue

        # Format the Technique Value: "TXXXX - Technique Name"
        try:
            formatted_technique_id_full = str(technique_id_full).upper()
            technique_value = f"{formatted_technique_id_full} - {str(technique_name)}"
        except Exception as e:
            print(f"Warning: Could not format technique value for item {item}. Error: {e}")
            continue

        # Add the technique to the list for the corresponding tactic, avoiding duplicates
        if technique_value not in attack_chain_data[tactic_key]:
            attack_chain_data[tactic_key].append(technique_value)

    # Convert defaultdict to a regular dict for the final output (optional, but common)
    return dict(attack_chain_data)

This is an export of the ATT&CK data from Crowdstrike's threat intelligence platform. Crowdstrike supplies this in a format for MITRE ATT&CK Navigator overlays, but we can repurpose it to our use case here by processing it with `generate_attack_chain_data`, defined above.

In [ ]:
actor_data = [
  {
    "id": "ta0001_t1078.001",
    "tactic_id": "ta0001",
    "tactic_name": "Initial Access",
    "technique_id": "t1078.001",
    "technique_name": "Default Accounts",
    "observables": [
      "Access to victims via compromised accounts"
    ]
  },
  {
    "id": "ta0001_t1133",
    "tactic_id": "ta0001",
    "tactic_name": "Initial Access",
    "technique_id": "t1133",
    "technique_name": "External Remote Services",
    "observables": [
      "Use of VPN services"
    ]
  },
  {
    "id": "ta0001_t1189",
    "tactic_id": "ta0001",
    "tactic_name": "Initial Access",
    "technique_id": "t1189",
    "technique_name": "Drive-by Compromise",
    "reports": [
      "CSMR-20006"
    ],
    "observables": [
      "Unspecified use of drive-by compromise"
    ]
  },
  {
    "id": "ta0001_t1190",
    "tactic_id": "ta0001",
    "tactic_name": "Initial Access",
    "technique_id": "t1190",
    "technique_name": "Exploit Public-Facing Application",
    "observables": [
      "Exploitation of unidentified public-facing applications"
    ]
  },
  {
    "id": "ta0001_t1566.001",
    "tactic_id": "ta0001",
    "tactic_name": "Initial Access",
    "technique_id": "t1566.001",
    "technique_name": "Spearphishing Attachment",
    "observables": [
      "KRYPTONITE PANDA has reportedly used spear phishing messages to deliver weaponized document files"
    ]
  },
  {
    "id": "ta0001_t1566.002",
    "tactic_id": "ta0001",
    "tactic_name": "Initial Access",
    "technique_id": "t1566.002",
    "technique_name": "Spearphishing Link",
    "observables": [
      "KRYPTONITE PANDA may use malicious hyperlinks to weaponized document files "
    ]
  },
  {
    "id": "ta0002_t1047",
    "tactic_id": "ta0002",
    "tactic_name": "Execution",
    "technique_id": "t1047",
    "technique_name": "Windows Management Instrumentation",
    "observables": [
      "KRYPTONITE PANDA has used WMI based scripts to deploy GreenCrash RAT. The WMI script mof.txt configures a persistence mechanism using an event-consumer binding. The event is a timer set to five minute intervals, and the consumer is a JavaScript function that will execute the command regsvr32 /s %systemroot%\\conhost.dll"
    ]
  },
  {
    "id": "ta0002_t1059",
    "tactic_id": "ta0002",
    "tactic_name": "Execution",
    "technique_id": "t1059",
    "technique_name": "Command and Scripting Interpreter",
    "observables": [
      "KRYPTONITE PANDA GreenCrash RAT provides remote shell functionality either by:\ncopying cmd.exe to %TEMP%\\svchost.exe and using this to execute received commands - or - copying cmd.exe to %WINDIR%\\Temp\\system and executing it as remote shell\nKRYPTONITE PANDA has used command line interaction to execute GreenCrash RAT loader executables"
    ]
  },
  {
    "id": "ta0002_t1059.001",
    "tactic_id": "ta0002",
    "tactic_name": "Execution",
    "technique_id": "t1059.001",
    "technique_name": "PowerShell",
    "reports": [
      "CSA-17205",
      "CSA-200976",
      "CSMR-20008",
      "CSWR-19046"
    ],
    "observables": [
      "\"KRYPTONITE PANDA has used VBA macros contained within .dotm files to load the DADSTACHE and DADJOKE implants. Malicious macros have Decoded base64-encoded URLs hosting a second-stage downloaders and used the URLDownloadToFileA function to retrieve externally hosted content. Retrieved content has been written to the %APPDATA%\\Microsoft\\Office\\ directory\nKRYPTONITE PANDA has used WMI based scripts to deploy GreenCrash RAT\nKRYPTONITE PANDA has used Visual basic macros to deploy secondary payloads\nUsed PowerShell"
    ]
  },
  {
    "id": "ta0002_t1106",
    "tactic_id": "ta0002",
    "tactic_name": "Execution",
    "technique_id": "t1106",
    "technique_name": "Native API",
    "observables": [
      "KRYPTONITE PANDA has used the Scripting.FileSystemObject function to initialize a DLL search order hijacking chain"
    ]
  },
  {
    "id": "ta0002_t1203",
    "tactic_id": "ta0002",
    "tactic_name": "Execution",
    "technique_id": "t1203",
    "technique_name": "Exploitation for Client Execution",
    "observables": [
      "KRYPTONITE PANDA has targeted known client software vulnerabilities including CVE-2017-0199, CVE-2017-8759, and CVE-2018-0802"
    ]
  },
  {
    "id": "ta0002_t1204.002",
    "tactic_id": "ta0002",
    "tactic_name": "Execution",
    "technique_id": "t1204.002",
    "technique_name": "Malicious File",
    "observables": [
      "KRYPTONITE PANDA has used weaponized malicious documents which require user interaction in order to deploy additional payloads\nKRYPTONITE PANDA has used weaponized malicious documents with embedded Visual Basic macros"
    ]
  },
  {
    "id": "ta0003_t1053.002",
    "tactic_id": "ta0003",
    "tactic_name": "Persistence",
    "technique_id": "t1053.002",
    "technique_name": "At",
    "observables": [
      "KRYPTONITE PANDA has used at.exe and schtasks.exe to persist GreenCrash RAT on compromised systems"
    ]
  },
  {
    "id": "ta0003_t1053.005",
    "tactic_id": "ta0003",
    "tactic_name": "Persistence",
    "technique_id": "t1053.005",
    "technique_name": "Scheduled Task",
    "observables": [
      "KRYPTONITE PANDA has used scheduleded tasks with the filename AdobeSvc to execute legitimate executables as part of DLL search order hijacking "
    ]
  },
  {
    "id": "ta0003_t1505.003",
    "tactic_id": "ta0003",
    "tactic_name": "Persistence",
    "technique_id": "t1505.003",
    "technique_name": "Web Shell",
    "observables": [
      "KRYPTONITE PANDA has used multiple webshells including China Chopper and Angel Shell on compromised web servers for persistence\nKRYPTONITE PANDA has used a JScript evaluator webshell with the filename app_offline.DISABLED.aspx"
    ]
  },
  {
    "id": "ta0003_t1574.001",
    "tactic_id": "ta0003",
    "tactic_name": "Persistence",
    "technique_id": "t1574.001",
    "technique_name": "DLL Search Order Hijacking"
  },
  {
    "id": "ta0003_t1574.002",
    "tactic_id": "ta0003",
    "tactic_name": "Persistence",
    "technique_id": "t1574.002",
    "technique_name": "DLL Side-Loading",
    "reports": [
      "CSA-250446"
    ],
    "observables": [
      "KRYPTONITE PANDA has used legitimate applications to side-load malicious DLLs"
    ]
  },
  {
    "id": "ta0005_t1027",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1027",
    "technique_name": "Obfuscated Files or Information",
    "observables": [
      "KRYPTONITE PANDA has used AES-128 to encode a DADSTACHE payload\nKRYPTONITE PANDA has used Base64 encoding to obfuscate VBScript loaders\nKRYPTONITE PANDA has obfuscated GreenCrash RAT payloads using XOR\nKRYPTONITE PANDA has used the single-byte XOR key 0xD7 to obfuscate downloader configurations"
    ]
  },
  {
    "id": "ta0005_t1027.002",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1027.002",
    "technique_name": "Software Packing",
    "observables": [
      "KRYPTONITE PANDA has used the .NET packer ConfuserEx to obfuscate .NET binaries"
    ]
  },
  {
    "id": "ta0005_t1027.003",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1027.003",
    "technique_name": "Steganography",
    "observables": [
      "Stored stolen data on GitHub disguised as benign images"
    ]
  },
  {
    "id": "ta0005_t1027.004",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1027.004",
    "technique_name": "Compile After Delivery",
    "observables": [
      "KRYPTONITE PANDA has used Base64 encoded second stage loaders run after retrieval on host"
    ]
  },
  {
    "id": "ta0005_t1036",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1036",
    "technique_name": "Masquerading",
    "observables": [
      "KRYPTONITE PANDA has renamed DADSTACHE DLLs using the filename LogiMail.dll\nKRYPTONITE PANDA has placed DADSTACHE implant files in the %APPDATA%\\Microsoft\\Office\\ directory\nKRYPTONITE PANDA has renamed DADJOKE DLLs using the filename mpsvc.dll\nKRYPTONITE PANDA has placed a GreenCrash loader in the C:\\Temp\\dell\\ directory\nKRYPTONITE PANDA has placed implant files in the c:\\ProgramData\\VMware\\logs directory"
    ]
  },
  {
    "id": "ta0005_t1070.004",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1070.004",
    "technique_name": "File Deletion",
    "observables": [
      "KRYPTONITE PANDA JJDOOR dropper delpys a secondary VBScript which contains functionality to delete installer files"
    ]
  },
  {
    "id": "ta0005_t1070.006",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1070.006",
    "technique_name": "Timestomp",
    "observables": [
      "KRYPTONITE PANDA multitool contains functionality to modify file timestamps"
    ]
  },
  {
    "id": "ta0005_t1140",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1140",
    "technique_name": "Deobfuscate/Decode Files or Information",
    "observables": [
      "KRYPTONITE PANDA has used VBA macros to decode base-64 encoded URLs hosting second stage content\nKRYPTONITE PANDA has used an AES-128 key based on the sample SHA256 hash to decode a DADSTACHE payload\nKRYPTONITE PANDA has used Base64 to decode VBScript loaders\nKRYPTONITE PANDA has  decoded GreenCrash RAT payload files with the XOR key 43 72 CD 1D 01 65 9F 7A D0 47 65 1D 9A 60 3C 5F\nKRYPTONITE PANDA GreenCrash RAT loader uses XXTEA algorithm to decode a configuration file"
    ]
  },
  {
    "id": "ta0005_t1218.005",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1218.005",
    "technique_name": "Mshta",
    "observables": [
      "KRYPTONITE PANDA has used malicious HTA files to load secondary payloads including Cobalt Strike and JJDOOR"
    ]
  },
  {
    "id": "ta0005_t1221",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1221",
    "technique_name": "Template Injection",
    "observables": [
      "KRYPTONITE PANDA has used weaponized Microsoft Office document which take advantage of the .docx hyperlink feature to retrieve and execute additional payloads"
    ]
  },
  {
    "id": "ta0005_t1574.001",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1574.001",
    "technique_name": "DLL Search Order Hijacking",
    "observables": [
      "KRYPTONITE PANDA has used search order hijacking in order to execute DADSTACHE DLL payloads\nKRYPTONITE PANDA has used search order hijacking to execute a custom keylogger via the legitimate executable debug.exe, and the DLL dbgeng.dll"
    ]
  },
  {
    "id": "ta0005_t1620",
    "tactic_id": "ta0005",
    "tactic_name": "Defense Evasion",
    "technique_id": "t1620",
    "technique_name": "Reflective Code Loading",
    "reports": [
      "CSA-250446"
    ],
    "observables": [
      "KRYPTONITE PANDA likely loaded .NET assemblies for in-memory execution"
    ]
  },
  {
    "id": "ta0006_t1003.003",
    "tactic_id": "ta0006",
    "tactic_name": "Credential Access",
    "technique_id": "t1003.003",
    "technique_name": "NTDS",
    "reports": [
      "CSA-250446"
    ],
    "observables": [
      "KRYPTONITE PANDA obtained the AD domain database NTDS.dit to access credential hashes"
    ]
  },
  {
    "id": "ta0006_t1040",
    "tactic_id": "ta0006",
    "tactic_name": "Credential Access",
    "technique_id": "t1040",
    "technique_name": "Network Sniffing",
    "observables": [
      "KRYPTONITE PANDA deployed the publicly available Responder credential sniffing tool, with the filename Intel.exe"
    ]
  },
  {
    "id": "ta0007_t1016",
    "tactic_id": "ta0007",
    "tactic_name": "Discovery",
    "technique_id": "t1016",
    "technique_name": "System Network Configuration Discovery",
    "observables": [
      "KRYPTONITE PANDA GreenCrash RAT contains network enumeration functionality"
    ]
  },
  {
    "id": "ta0007_t1046",
    "tactic_id": "ta0007",
    "tactic_name": "Discovery",
    "technique_id": "t1046",
    "technique_name": "Network Service Discovery",
    "observables": [
      "KRYPTONITE PANDA has used a freely available PHP based port scanning tool\nKRYPTONITE PANDA multitool contains network port scanning functionality"
    ]
  },
  {
    "id": "ta0007_t1082",
    "tactic_id": "ta0007",
    "tactic_name": "Discovery",
    "technique_id": "t1082",
    "technique_name": "System Information Discovery",
    "observables": [
      "KRYPTONITE PANDA JJDOOR captures host enumeration data\nKRYPTONITE PANDA GreenCrash RAT contains host enumeration functionality"
    ]
  },
  {
    "id": "ta0008_t1021.002",
    "tactic_id": "ta0008",
    "tactic_name": "Lateral Movement",
    "technique_id": "t1021.002",
    "technique_name": "SMB/Windows Admin Shares",
    "observables": [
      "KRYPTONITE PANDA multitool contains functionality to administer Windows administrative functions over SMB"
    ]
  },
  {
    "id": "ta0009_t1056.001",
    "tactic_id": "ta0009",
    "tactic_name": "Collection",
    "technique_id": "t1056.001",
    "technique_name": "Keylogging",
    "observables": [
      "KRYPTONITE PANDA deployed a custom keylogger tool with the filename dbgeng.dll "
    ]
  },
  {
    "id": "ta0009_t1074.001",
    "tactic_id": "ta0009",
    "tactic_name": "Collection",
    "technique_id": "t1074.001",
    "technique_name": "Local Data Staging",
    "observables": [
      "Data staged in unspecified locations for data exfil"
    ]
  },
  {
    "id": "ta0009_t1074.002",
    "tactic_id": "ta0009",
    "tactic_name": "Collection",
    "technique_id": "t1074.002",
    "technique_name": "Remote Data Staging",
    "observables": [
      "Data moved from compromised hosts to central host prior to exfil"
    ]
  },
  {
    "id": "ta0009_t1560",
    "tactic_id": "ta0009",
    "tactic_name": "Collection",
    "technique_id": "t1560",
    "technique_name": "Archive Collected Data",
    "observables": [
      "Archived data for exfiltration"
    ]
  },
  {
    "id": "ta0010_t1041",
    "tactic_id": "ta0010",
    "tactic_name": "Exfiltration",
    "technique_id": "t1041",
    "technique_name": "Exfiltration Over C2 Channel",
    "observables": [
      "Unspecified use"
    ]
  },
  {
    "id": "ta0010_t1567.002",
    "tactic_id": "ta0010",
    "tactic_name": "Exfiltration",
    "technique_id": "t1567.002",
    "technique_name": "Exfiltration to Cloud Storage",
    "reports": [
      "CSIT-22252"
    ],
    "observables": [
      "KRYPTONITE PANDA exfiltrated stolen data to GitHub and Dropbox "
    ]
  },
  {
    "id": "ta0011_t1001.003",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1001.003",
    "technique_name": "Protocol Impersonation",
    "observables": [
      "KRYPTONITE PANDA JJDOOR implant uses Base64 to encode content within command and control traffic"
    ]
  },
  {
    "id": "ta0011_t1043",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1043",
    "technique_name": "Commonly Used Port",
    "observables": [
      "KRYPTONITE PANDA Green Rash RAT has used TCP ports 443 and 8080 for command and control communications"
    ]
  },
  {
    "id": "ta0011_t1071.001",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1071.001",
    "technique_name": "Web Protocols",
    "observables": [
      "KRYPTONITE PANDA JJDOOR implant uses HTTP for command and control communications"
    ]
  },
  {
    "id": "ta0011_t1095",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1095",
    "technique_name": "Non-Application Layer Protocol",
    "observables": [
      "KRYPTONITE PANDA GreenCrash RAT uses the Windows Socket Library to send and receive a PLib8-compressed data messages between the RAT and the C2 server."
    ]
  },
  {
    "id": "ta0011_t1102",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1102",
    "technique_name": "Web Service",
    "observables": [
      "KRYPTONITE PANDA has used Simple Object Access Protocol (SOAP) to retrieve additional payloads"
    ]
  },
  {
    "id": "ta0011_t1102.001",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1102.001",
    "technique_name": "Dead Drop Resolver",
    "observables": [
      "KRYPTONITE PANDA has used wordpress services as dead drop resolvers for GreenCrash RAT tasking data"
    ]
  },
  {
    "id": "ta0011_t1105",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1105",
    "technique_name": "Ingress Tool Transfer",
    "observables": [
      "KRYPTONITE PANDA has used the URLDownloadToFileA function to retrieve externally hosted content\nKRYPTONITE PANDA has used Simple Object Access Protocol (SOAP) to retrieve additional payloads\nKRYPTONITE PANDA has used a custom downloader with the filename rcdll.dll to retrieve external payloads\nKRYPTONITE PANDA has used a custom downloader JavaScript downloader with the filename 0.js"
    ]
  },
  {
    "id": "ta0011_t1571",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1571",
    "technique_name": "Non-Standard Port",
    "observables": [
      "KRYPTONITE PANDA Green Rash RAT has used TCP port 8084 for command and control communications"
    ]
  },
  {
    "id": "ta0011_t1572",
    "tactic_id": "ta0011",
    "tactic_name": "Command and Control",
    "technique_id": "t1572",
    "technique_name": "Protocol Tunneling",
    "observables": [
      "Unspecified use"
    ]
  },
  {
    "id": "ta0042_t1583.001",
    "tactic_id": "ta0042",
    "tactic_name": "Resource Development",
    "technique_id": "t1583.001",
    "technique_name": "Domains",
    "observables": [
      "Creation of typosquatting domains",
      "KRYPTONITE PANDA has registered domain names relevant to target scope for use as command-and-control and hosting infrastructure including: \nairbusocean[.]com, teledynegroup[.]com, scsnewstoday[.]com, www.thyssenkrupp-marinesystems[.]org"
    ]
  },
  {
    "id": "ta0042_t1583.002",
    "tactic_id": "ta0042",
    "tactic_name": "Resource Development",
    "technique_id": "t1583.002",
    "technique_name": "DNS Server",
    "observables": [
      "KRYPTONITE PANDA employs multiple DNS name servers as part of infrastructure management including:\nGoDaddy, Namecheap, CloudFlare"
    ]
  },
  {
    "id": "ta0042_t1583.003",
    "tactic_id": "ta0042",
    "tactic_name": "Resource Development",
    "technique_id": "t1583.003",
    "technique_name": "Virtual Private Server",
    "reports": [
      "CSA-250446"
    ],
    "observables": [
      "KRYPTONITE PANDA typically hosts VPS infrastructure at Vultr (AS20473) and Digital Ocean (AS14061)"
    ]
  },
  {
    "id": "ta0042_t1587.003",
    "tactic_id": "ta0042",
    "tactic_name": "Resource Development",
    "technique_id": "t1587.003",
    "technique_name": "Digital Certificates",
    "observables": [
      "KRYPTONITE PANDA has used self signed certificates with the subject and issuer field set to C=US, ST=Texas, L=Austin, O=Development, CN=localhost"
    ]
  },
  {
    "id": "ta0043_t1589.001",
    "tactic_id": "ta0043",
    "tactic_name": "Reconnaissance",
    "technique_id": "t1589.001",
    "technique_name": "Credentials",
    "observables": [
      "Stole credentials"
    ]
  }
]

Using the processed attack chain data, generate an ASOM for KRYPTONITE PANDA. This code exports both  the full ASOM where all rows are distinct, better suited for further machine processing, as well as the finished ASOM where like cells are merged to create the nested structure depicted in TC 3-12.2.4.1.

In [ ]:
attack_chain_data = generate_attack_chain_data(actor_data)

log("Building ASOM...")
resulting_asom = build_asom(attack_chain_data)
log("Finished.")

formatted_df = format_asom(resulting_asom)

representative_ccirs_map = get_representative_ccirs_by_tactic_simplified(formatted_df)

# Perform the update and grouping
updated_df = update_ccirs(formatted_df, representative_ccirs_map)

updated_df = update_sequential_ccir_index(updated_df)
updated_df = update_hierarchical_indicator_index(updated_df)
updated_df = update_final_evidence_index(updated_df, indicator_index_col = "Indicator Index", output_evidence_index_col = "Evidence Index")

# Export the full ASOM
updated_df.to_excel("kryptonite-panda_full.xlsx")

final_df = create_visually_spanned_df(updated_df)

print("final_df (with MultiIndex for visual spanning):")
# Displaying the DataFrame with to_string() often shows the MultiIndex spanning.
# In Jupyter notebooks, just `display(final_df)` or `final_df` would also work well.
with pd.option_context('display.max_rows', None, # Show all rows to see full effect
                        'display.width', 1200,    # Adjust width as needed
                        'display.max_colwidth', 30,
                        'display.expand_frame_repr', False # Prevent wrapping if too wide for console
                        ):
    # display(final_df)
    display(final_df.filter(items=["CCIR Index", "CCIR", "Indicator Index", "Indicator", "Evidence Index", "Evidence", "Data", "NAI", "Action"]))

In [ ]:
# Export the merged ASOM
final_df.to_excel("kryptonite-panda_merged.xlsx")